In [98]:
import cv2
from ultralytics import YOLO

In [131]:
# path = "/home/hassaan/Downloads/DPT2/343.png"
path = "/home/hassaan/Downloads/allFinal/316.png"

In [100]:
model = YOLO("models/yolov8m.pt")

In [113]:
def return_people_bboxes(path):
    frame = cv2.imread(path)

    results = model(frame)
    bboxes = []
    confs = []
    # Render the results
    for result in results:
        if result.boxes is not None:
            for box in result.boxes:
                confidence = float(box.conf[0])
                bbox = box.xyxy[0].numpy().astype(int)

                if int(box.cls[0]) == 0:
                    bboxes.append(list(bbox))
                    confs.append(confidence)

                    #     print(bbox)
                    #     bboxes.append(bbox)
                    x1, y1, x2, y2 = bbox

                    # Draw bounding box and label
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(
                        frame,
                        f"{0} {confidence:.2f}",
                        (x1, y1 + 10),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.5,
                        (0, 255, 0),
                        2,
                    )
    # print(frame)
    return bboxes, confs, frame

In [114]:
bboxes, confs, frame = return_people_bboxes(path)


0: 384x640 2 persons, 2 bicycles, 1 car, 916.7ms
Speed: 5.2ms preprocess, 916.7ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)


In [116]:
bboxes.sort(key=lambda x: x[0])
bboxes

[[203, 234, 384, 668], [504, 222, 677, 580]]

In [47]:
def check_overlap(box1, box2):
    # bbox is TL and BR corner coordinates
    x1, y1, x2, y2 = box1
    a1, b1, a2, b2 = box2

    box1_corners = [
        (x1, y1),  # TL
        (x2, y1),  # TR
        (x1, y2),  # BL
        (x2, y2),  # BR
    ]

    while False:
        corner_overlap_areas = {
            "TL": ((a2 - x1) * (b2 - y1)),
            "TR": ((x1 - a1) * (b2 - y2)),
            "BL": ((a2 - x1) * (y2 - b1)),
            "BR": ((x2 - a1) * (y2 - b1)),
        }

    overlapping_corners = 0
    single_corener = () # returns corner for single corner case with small overlap
    for corner in box1_corners:
        x, y = corner
        # check if x1,y1 is in box2
        if (x >= a1 and x <= a2) and (y >= b1 and y <= b2):
            overlapping_corners += 1
            single_corener = (x,y)

    area = 0
    if overlapping_corners > 0:
        # calculate the coordinates of the overlapping rectangle
        x_left = max(x1, a1)
        y_top = max(y1, b1)
        x_right = min(x2, a2)
        y_bottom = min(y2, b2)

        # calculate overlap area
        overlap_width = x_right - x_left
        overlap_height = y_bottom - y_top
        area = overlap_width * overlap_height

    return overlapping_corners, area

In [ ]:
def image_to_dataset(frame,bboxes,confs,index):
    
    # 1 person
    if len(bboxes) == 1:
        # write depthmap of corresponding image as it is into dataset
        section1 = frame
        pass
    
    # 2 person
    if len(bboxes) == 2:

        bbox1, bbox2 = bboxes
        overlapping_corners , area = check_overlap(bbox1, bbox2)
        
        # no overlap
        if area == 0:
            # sort bboxes to organize by TL corner
            bboxes.sort(key= lambda x: x[0])
            box1,box2 = bboxes
            x1, y1, x2 ,y2 = box1
            a1, b1, a2, b2 = box2
            
            # not right, above or below. :. box1 is to the left of box2 
            if x2 <= a1:
                mid_x = (x2 + a1) // 2
                
                section1 = frame[:mid_x,:]
                section2 = frame[mid_x:,:]
            
            # box1 is above or below box2
            else:
                if y2 >= b1:    #if above
                    mid_y = (y2 + b1) // 2
                    
                elif b2 >= y1:  #if below
                    mid_y = (y1 + b2) // 2
                    
                section1 = frame[:,:mid_y]
                section2 = frame[:,mid_y:]
                 
        
        # overlap
        if area > 0:
            # 1 corner overlap
            if overlapping_corners == 1:
                a1, a2, b1, b2 = bboxes[1]
                
                # if overlap area is less than 30% area of bbox2
                if area < 0.3 * (a2-a1) * (b2-b1):
                    bboxes.sort(key= lambda x: x[0])
                    box1, box2 = bboxes
                    
                    mid_x = (box1[2] + box2[0]) / 2
                    
                    section1 = frame[:mid_x,:]
                    section2 = frame[mid_x:,:]
                
                else:
                    
                    # box1 is significantly more recognizable than box2
                    if confs[0] > confs[1] - 10:
                        # use only box1
                        section1 = frame[box1[0]:box1[2],box1[1]:box1[3]]
                        
                    # box2 is significantly more recognizable than box1
                    elif confs[0] < confs[1] - 10:
                        # use only box1
                        section1 = frame[box2[0]:box2[2],box2[1]:box2[3]]
                        
                    else:
                        # both box are similarly recognizable and have significant overlap
                        section1 = frame[box1[0]:box1[2],box1[1]:box1[3]]
                        section2 = frame[box2[0]:box2[2],box2[1]:box2[3]]
                        
            
            # edge overlap
            elif overlapping_corners == 2:
                
                pass
                
            
            # full overlap
            else:
                pass

    # 3 person
    if len(bboxes) > 2:
        pass
    
    if bboxes == []:
        # skip image
        pass
    pass

In [60]:
bboxes.sort(key=lambda x: x[0])
bboxes

[[388, 231, 631, 767], [605, 199, 799, 683]]

In [61]:
frame = cv2.imread(path)

In [118]:
bboxes.sort(key=lambda x: x[0])
box1, box2 = bboxes

mid_x = (box1[2] + box2[0]) // 2

section1 = frame[:, :mid_x]
section2 = frame[:, mid_x:]

while True:
    cv2.imshow("1", section1)
    cv2.imshow("2", section2)

    if cv2.waitKey(10) & 0xFF == ord("q"):
        break
cv2.destroyAllWindows()